In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import datetime
from datetime import date

## Resources

In [2]:
# Today price: https://selectra.es/energia/info/que-es/precio-kwh
# Tomorrow price: https://selectra.es/energia/info/que-es/precio-kwh#precio-luz-manana
# MoM evolution: https://selectra.es/energia/info/que-es/precio-kwh#precio-luz-hoy

# Overall metrics: https://tarifaluzhora.es/
# Electro devices per year: https://tarifaluzhora.es/info/consumo-mensual-luz#como-reparte-consumo

# Consumption per device per year: https://comparadorluz.com/faq/consumo-medio-casa#consumo-luz-electrodomesticos
# Consumption per device: https://comparadorluz.com/faq/consumo-medio-casa#consumo-luz-electrodomesticos

## Scraping today's prices from https://tarifaluzhora.es/

In [3]:
    today = date.today().strftime('%d-%m-%Y')
    
    url = 'https://tarifaluzhora.es/'

    headers= {'Accept-Encoding':'gzip, deflate',
              'Accept-Language':'en-US,en;q=0.9',
              'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}

    response = requests.get(url, headers = headers)
    response.status_code

    soup = BeautifulSoup(response.content)
    #print(soup.prettify())

    average_today_price = soup.find('div',attrs={'class':'inner_block gauge_day'})
    average_today_price = average_today_price.span.string[:6]

    cheapest_today_time = soup.find('div',attrs={'class':'inner_block gauge_low'})
    cheapest_today_time = cheapest_today_time.span.string.replace('h','')

    expensive_today_time = soup.find('div',attrs={'class':'inner_block gauge_hight'})
    expensive_today_time = expensive_today_time.span.string.replace('h','')

    lowest_today_price = soup.find('span',attrs={'class':'sub_text green'})
    lowest_today_price= lowest_today_price.get_text().replace('\t','').replace('\n','')[:7]

    highest_today_price = soup.find('span',attrs={'class':'sub_text red'})
    highest_today_price= highest_today_price.get_text().replace('\t','').replace('\n','')[:7]

    print(f'Today is {today}')
    print('----------------------------------------------------------------------')
    print(f'Average price of today is: {average_today_price}€ per kwh')
    print(f'Lowest price of today is: {lowest_today_price}€ per kwh')
    print(f'Highest price of today is: {highest_today_price}€ per kwh')
    print('----------------------------------------------------------------------')
    print(f'Cheapest time of today was between {cheapest_today_time} hs')
    print(f'Expensive time of today was between {expensive_today_time} hs')

    hour_range = soup.find_all('span',attrs={'itemprop':'description'})
    hour_price = soup.find_all('span',attrs={'itemprop':'price'})

    hours = []

    for i in hour_range:
        hours.append(i.string.replace(':','').replace('h',''))

    prices = []

    for i in hour_price:
        prices.append(i.string.replace('€/kW','').replace('h','').strip())
        
    price_per_hour = pd.DataFrame()  
    price_per_hour['Hours'] = hours
    price_per_hour['Prices'] = prices
    price_per_hour['Prices'] = price_per_hour['Prices'].astype(float)
    price_per_hour['Avg Today'] = price_per_hour['Prices'].mean()
    price_per_hour['Deviation'] = price_per_hour['Prices'].mean() - price_per_hour['Prices']

    def price_flag(x):
        if x<0:
            return 'Expensive'
        elif x>0: 
            return 'Cheaper'
        else:
            return 'Regular'

    price_per_hour['Price Flag'] = price_per_hour['Deviation'].apply(price_flag)
    price_per_hour

Today is 04-05-2022
----------------------------------------------------------------------
Average price of today is: 0.2840€ per kwh
Lowest price of today is: 0.25112€ per kwh
Highest price of today is: 0.33504€ per kwh
----------------------------------------------------------------------
Cheapest time of today was between 01 - 02 hs
Expensive time of today was between 21 - 22 hs


,Hours,Prices,Avg Today,Deviation,Price Flag
0,00 - 01,0.25396,0.284044,0.030084,Cheaper
1,01 - 02,0.25112,0.284044,0.032924,Cheaper
2,02 - 03,0.25421,0.284044,0.029834,Cheaper
3,03 - 04,0.25772,0.284044,0.026324,Cheaper
4,04 - 05,0.25879,0.284044,0.025254,Cheaper
5,05 - 06,0.25818,0.284044,0.025864,Cheaper
6,06 - 07,0.25714,0.284044,0.026904,Cheaper
7,07 - 08,0.25953,0.284044,0.024514,Cheaper
8,08 - 09,0.29176,0.284044,-0.007716,Expensive
9,09 - 10,0.27355,0.284044,0.010494,Cheaper


In [4]:
def scrape_today():
    
    today = date.today().strftime('%d-%m-%Y')

    url = 'https://tarifaluzhora.es/'

    headers= {'Accept-Encoding':'gzip, deflate',
              'Accept-Language':'en-US,en;q=0.9',
              'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}

    response = requests.get(url, headers = headers)
    response.status_code

    soup = BeautifulSoup(response.content)
    #print(soup.prettify())

    average_today_price = soup.find('div',attrs={'class':'inner_block gauge_day'})
    average_today_price = average_today_price.span.string[:6]

    cheapest_today_time = soup.find('div',attrs={'class':'inner_block gauge_low'})
    cheapest_today_time = cheapest_today_time.span.string.replace('h','')

    expensive_today_time = soup.find('div',attrs={'class':'inner_block gauge_hight'})
    expensive_today_time = expensive_today_time.span.string.replace('h','')

    lowest_today_price = soup.find('span',attrs={'class':'sub_text green'})
    lowest_today_price= lowest_today_price.get_text().replace('\t','').replace('\n','')[:7]

    highest_today_price = soup.find('span',attrs={'class':'sub_text red'})
    highest_today_price= highest_today_price.get_text().replace('\t','').replace('\n','')[:7]

    print(f'Today is {today}')
    print('----------------------------------------------------------------------')
    print(f'Average price of today is: {average_today_price}€ per kwh')
    print(f'Lowest price of today is: {lowest_today_price}€ per kwh')
    print(f'Highest price of today is: {highest_today_price}€ per kwh')
    print('----------------------------------------------------------------------')
    print(f'Cheapest time of today was between {cheapest_today_time} hs')
    print(f'Expensive time of today was between {expensive_today_time} hs')

    hour_range = soup.find_all('span',attrs={'itemprop':'description'})
    hour_price = soup.find_all('span',attrs={'itemprop':'price'})

    hours = []

    for i in hour_range:
        hours.append(i.string.replace(':','').replace('h',''))

    prices = []

    for i in hour_price:
        prices.append(i.string.replace('€/kW','').replace('h','').strip())
        
    price_per_hour = pd.DataFrame()  
    price_per_hour['Hours'] = hours
    price_per_hour['Prices'] = prices
    price_per_hour['Prices'] = price_per_hour['Prices'].astype(float)
    price_per_hour['Avg Today'] = price_per_hour['Prices'].mean()
    price_per_hour['Deviation'] = price_per_hour['Prices'].mean() - price_per_hour['Prices']

    def price_flag(x):
        if x<0:
            return 'Expensive'
        elif x>0: 
            return 'Cheaper'
        else:
            return 'Regular'

    price_per_hour['Price Flag'] = price_per_hour['Deviation'].apply(price_flag)

    def light_consumption(x):
        return x*0.117

    def kitchen_consumption(x):
        return x*0.093

    def hot_water_consumption(x):
        return x*0.075

    def heat_consumption(x):
        return x*0.074

    def refrigeration_consumption(x):
        return x*0.023

    def standby_consumption(x):
        return x*0.066

    def home_appliances_consumption(x):
        return x*0.552

    price_per_hour['Home Appliances'] = price_per_hour['Prices'].apply(home_appliances_consumption)
    price_per_hour['Light'] = price_per_hour['Prices'].apply(light_consumption)
    price_per_hour['Kitchen'] = price_per_hour['Prices'].apply(kitchen_consumption)
    price_per_hour['Hot water'] = price_per_hour['Prices'].apply(hot_water_consumption)
    price_per_hour['Heat'] = price_per_hour['Prices'].apply(heat_consumption)
    price_per_hour['Refrigeration'] = price_per_hour['Prices'].apply(refrigeration_consumption)
    price_per_hour['Stand By'] = price_per_hour['Prices'].apply(standby_consumption)

    return price_per_hour

## Scraping tomorrow's prices from https://tarifaluzhora.es/info/precio-kwh-manana

If you re run this block and see 0 values, create another block and run this instead:  
*tomorrow_prices_df_backup = pd.read_csv('tomorrow_data_for_testing.csv')*

Note that some variables and code should be updated with this df instead of the preovious one.

In [3]:
#tomorrow_prices_df_backup = pd.read_csv('tomorrow_data_for_testing.csv')
#tomorrow_prices_df_backup

In [5]:
now = datetime.datetime.now().strftime("%I:%M %p")
#Backup comment as a reminder
print(f'Its {now} right now. Tomorrow prices for Spain can only be checked after 8:15 PM GMT+2. Come back later if you get zero values.')

url = 'https://tarifaluzhora.es/info/precio-kwh-manana'

headers= {'Accept-Encoding':'gzip, deflate',
              'Accept-Language':'en-US,en;q=0.9',
              'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}

response = requests.get(url, headers = headers)
response.status_code
soup = BeautifulSoup(response.content)


tomorrow_prices_df = pd.read_html(url)[0]
tomorrow_prices_df['Hora'] = tomorrow_prices_df['Hora'].str.replace("h","")
tomorrow_prices_df['Península,  Baleares y Canarias'] = tomorrow_prices_df['Península,  Baleares y Canarias'].str.replace('€/kWh','').replace(',','.')
tomorrow_prices_df['Península,  Baleares y Canarias'] = tomorrow_prices_df['Península,  Baleares y Canarias'].str.replace(',','.')
tomorrow_prices_df['Península,  Baleares y Canarias'] = tomorrow_prices_df['Península,  Baleares y Canarias'].str.strip()
tomorrow_prices_df['Ceuta y Melilla'] = tomorrow_prices_df['Ceuta y Melilla'].str.replace('€/kWh','').replace(',','.')
tomorrow_prices_df['Ceuta y Melilla'] = tomorrow_prices_df['Ceuta y Melilla'].str.replace(',','.')
tomorrow_prices_df['Ceuta y Melilla'] = tomorrow_prices_df['Ceuta y Melilla'].str.strip()
tomorrow_prices_df[['Península,  Baleares y Canarias','Ceuta y Melilla']] = tomorrow_prices_df[['Península,  Baleares y Canarias','Ceuta y Melilla']].astype(float)
tomorrow_prices_df = tomorrow_prices_df.rename(columns={'Hora':'Hour','Península,  Baleares y Canarias': 'Península, Baleares y Canarias (in €/kWh)', 'Ceuta y Melilla': 'Ceuta y Melilla (in €/kWh)'})
tomorrow_prices_df['Avg Today - Tier 1'] = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean()
tomorrow_prices_df['Deviation - Tier 1'] = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean() - tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)']
tomorrow_prices_df['Avg Today - Tier 2'] = tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean()
tomorrow_prices_df['Deviation - Tier 2'] = tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean() - tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean()
tomorrow_prices_df['Price Flag - Tier 1'] = tomorrow_prices_df['Deviation - Tier 1'].apply(price_flag)
tomorrow_prices_df['Price Flag - Tier 2'] = tomorrow_prices_df['Deviation - Tier 2'].apply(price_flag)
tomorrow_prices_df

Its 09:45 PM right now. Tomorrow prices for Spain can only be checked after 8:15 PM GMT+2. Come back later if you get zero values.


,Hour,"Península, Baleares y Canarias (in €/kWh)",Ceuta y Melilla (in €/kWh),Avg Today - Tier 1,Deviation - Tier 1,Avg Today - Tier 2,Deviation - Tier 2,Price Flag - Tier 1,Price Flag - Tier 2
0,00,0.24695,0.24695,0.271949,0.024999,0.273323,0.0,Cheaper,Regular
1,01,0.24182,0.24182,0.271949,0.030129,0.273323,0.0,Cheaper,Regular
2,02,0.23757,0.23757,0.271949,0.034379,0.273323,0.0,Cheaper,Regular
3,03,0.24725,0.24725,0.271949,0.024699,0.273323,0.0,Cheaper,Regular
4,04,0.25579,0.25579,0.271949,0.016159,0.273323,0.0,Cheaper,Regular
5,05,0.25450,0.25450,0.271949,0.017449,0.273323,0.0,Cheaper,Regular
6,06,0.25320,0.25320,0.271949,0.018749,0.273323,0.0,Cheaper,Regular
7,07,0.26144,0.29439,0.271949,0.010509,0.273323,0.0,Cheaper,Regular
8,08,0.29439,0.29439,0.271949,-0.022441,0.273323,0.0,Expensive,Regular
9,09,0.27778,0.27778,0.271949,-0.005831,0.273323,0.0,Expensive,Regular


In [6]:
def scrape_tomorrow():
    now = datetime.datetime.now().strftime("%I:%M %p")
    #Backup comment as a reminder
    print (f'Its {now} right now. Tomorrow prices for Spain can only be checked after 8:15 PM GMT+2. Come back later if you get zero values.')

    url = 'https://tarifaluzhora.es/info/precio-kwh-manana'

    headers= {'Accept-Encoding':'gzip, deflate',
              'Accept-Language':'en-US,en;q=0.9',
              'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.88 Safari/537.36'}

    response = requests.get(url, headers = headers)
    response.status_code
    soup = BeautifulSoup(response.content)


    tomorrow_prices_df = pd.read_html(url)[0]
    tomorrow_prices_df['Hora'] = tomorrow_prices_df['Hora'].str.replace("h","")
    tomorrow_prices_df['Península,  Baleares y Canarias'] = tomorrow_prices_df['Península,  Baleares y Canarias'].str.replace('€/kWh','').replace(',','.')
    tomorrow_prices_df['Península,  Baleares y Canarias'] = tomorrow_prices_df['Península,  Baleares y Canarias'].str.replace(',','.')
    tomorrow_prices_df['Península,  Baleares y Canarias'] = tomorrow_prices_df['Península,  Baleares y Canarias'].str.strip()
    tomorrow_prices_df['Ceuta y Melilla'] = tomorrow_prices_df['Ceuta y Melilla'].str.replace('€/kWh','').replace(',','.')
    tomorrow_prices_df['Ceuta y Melilla'] = tomorrow_prices_df['Ceuta y Melilla'].str.replace(',','.')
    tomorrow_prices_df['Ceuta y Melilla'] = tomorrow_prices_df['Ceuta y Melilla'].str.strip()
    tomorrow_prices_df[['Península,  Baleares y Canarias','Ceuta y Melilla']] = tomorrow_prices_df[['Península,  Baleares y Canarias','Ceuta y Melilla']].astype(float)
    tomorrow_prices_df = tomorrow_prices_df.rename(columns={'Hora':'Hour','Península,  Baleares y Canarias': 'Península, Baleares y Canarias (in €/kWh)', 'Ceuta y Melilla': 'Ceuta y Melilla (in €/kWh)'})
    tomorrow_prices_df['Avg Today - Tier 1'] = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean()
    tomorrow_prices_df['Deviation - Tier 1'] = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean() - tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)']
    tomorrow_prices_df['Avg Today - Tier 2'] = tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean()
    tomorrow_prices_df['Deviation - Tier 2'] = tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean() - tomorrow_prices_df['Ceuta y Melilla (in €/kWh)'].mean()
    tomorrow_prices_df['Price Flag - Tier 1'] = tomorrow_prices_df['Deviation - Tier 1'].apply(price_flag)
    tomorrow_prices_df['Price Flag - Tier 2'] = tomorrow_prices_df['Deviation - Tier 2'].apply(price_flag)

    #display(tomorrow_prices_df.describe())
    return tomorrow_prices_df

## Today vs tomorrow calculations

In [7]:
# Give expensive & cheap hours for tomorrow

expensive_hour = tomorrow_prices_df[tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'] == tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].max()][['Hour']]
expensive_hour_tomorrow = str(expensive_hour['Hour'])[:2]

cheapest_hour = tomorrow_prices_df[tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'] == tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].min()][['Hour']]
cheapest_hour_tomorrow = str(cheapest_hour['Hour'])[:2]

average_price_tomorrow = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean()
lowest_price_tomorrow = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].min()
highest_price_tomorrow = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].max()

# Comparing change average price, lowest and highest price, cheap and expensive hours -- today vs tomorrow

average_change_tomorrow_vs_today = (tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean()/price_per_hour['Prices'].mean())-1
lowest_change_tomorrow_vs_today = (tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].min()/price_per_hour['Prices'].min())-1
highest_change_tomorrow_vs_today = (tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].max()/price_per_hour['Prices'].max())-1

# Expensive and cheap hours for today and tomorrow 
expensive_hour_today = price_per_hour[price_per_hour['Prices'] == price_per_hour['Prices'].max()][['Hours']]
cheapest_hour_today = price_per_hour[price_per_hour['Prices'] == price_per_hour['Prices'].min()][['Hours']]

# Expensive and cheap hours tomorrow list
list_cheapest_hours_tomorrow = list(tomorrow_prices_df['Hour'][tomorrow_prices_df['Price Flag - Tier 1'] == 'Cheaper'])
cheapest_hours_tomorrow = ','.join(list_cheapest_hours_tomorrow)

list_expensive_hours_tomorrow = list(tomorrow_prices_df['Hour'][tomorrow_prices_df['Price Flag - Tier 1'] == 'Expensive'])
expensive_hours_tomorrow = ','.join(list_expensive_hours_tomorrow)

tomorrow = datetime.date.today() + datetime.timedelta(days=1)
tomorrow_date = tomorrow.strftime('%d-%m-%Y')

print(f'Today is {today}')
print('----------------------------------------------------------------------')
print(f'Average price of today is: {average_today_price}€ per kwh')
print(f'Lowest price of today is: {lowest_today_price}€ per kwh')
print(f'Highest price of today is: {highest_today_price}€ per kwh')
print('----------------------------------------------------------------------')
print(f'Cheapest time of today was between {cheapest_today_time} hs')
print(f'Expensive time of today was between {expensive_today_time} hs')
print('----------------------------------------------------------------------')
print(f'Tomorrow is {tomorrow_date}')
print('----------------------------------------------------------------------')
print(f'Average price for tomorrow will be: {average_price_tomorrow}€ per kwh')
print(f'Lowest price for tomorrow will be: {lowest_price_tomorrow}€ per kwh')
print(f'Highest price for tomorrow will be: {highest_price_tomorrow}€ per kwh')
print('----------------------------------------------------------------------')
print(f'The cheapest time for tomorrow will be {cheapest_hour_tomorrow} hs, while the cheapest hours for tomorrow will be: {cheapest_hours_tomorrow}')
print(f'Most expensive time for tomorrow will be {expensive_hour_tomorrow} hs, while the most xpensive hours for tomorrow will be: {expensive_hours_tomorrow}')
print('----------------------------------------------------------------------')
print(f'Today vs Tomorrow difference on the average price is {round(average_change_tomorrow_vs_today*100,2)}%')
print(f'Today vs Tomorrow difference on the lowest price is {round(lowest_change_tomorrow_vs_today*100,2)}%')
print(f'Today vs Tomorrow difference on the highest price is {round(highest_change_tomorrow_vs_today*100,2)}%')

Today is 04-05-2022
----------------------------------------------------------------------
Average price of today is: 0.2840€ per kwh
Lowest price of today is: 0.25112€ per kwh
Highest price of today is: 0.33504€ per kwh
----------------------------------------------------------------------
Cheapest time of today was between 01 - 02 hs
Expensive time of today was between 21 - 22 hs
----------------------------------------------------------------------
Tomorrow is 05-05-2022
----------------------------------------------------------------------
Average price for tomorrow will be: 0.27194875€ per kwh
Lowest price for tomorrow will be: 0.22875€ per kwh
Highest price for tomorrow will be: 0.34489€ per kwh
----------------------------------------------------------------------
The cheapest time for tomorrow will be 14 hs, while the cheapest hours for tomorrow will be: 00,01,02,03,04,05,06,07,14,15,16,17
Most expensive time for tomorrow will be 21 hs, while the most xpensive hours for tomorro

In [8]:
def today_tomorrow_comparison():

    # Give expensive & cheap hours for tomorrow

    expensive_hour = tomorrow_prices_df[tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'] == tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].max()][['Hour']]
    expensive_hour_tomorrow = str(expensive_hour['Hour'])[:2]

    cheapest_hour = tomorrow_prices_df[tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'] == tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].min()][['Hour']]
    cheapest_hour_tomorrow = str(cheapest_hour['Hour'])[:2]

    average_price_tomorrow = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean()
    lowest_price_tomorrow = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].min()
    highest_price_tomorrow = tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].max()

    # Comparing change average price, lowest and highest price, cheap and expensive hours -- today vs tomorrow

    average_change_tomorrow_vs_today = (tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].mean()/price_per_hour['Prices'].mean())-1
    lowest_change_tomorrow_vs_today = (tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].min()/price_per_hour['Prices'].min())-1
    highest_change_tomorrow_vs_today = (tomorrow_prices_df['Península, Baleares y Canarias (in €/kWh)'].max()/price_per_hour['Prices'].max())-1

    # Expensive and cheap hours for today and tomorrow 
    expensive_hour_today = price_per_hour[price_per_hour['Prices'] == price_per_hour['Prices'].max()][['Hours']]
    cheapest_hour_today = price_per_hour[price_per_hour['Prices'] == price_per_hour['Prices'].min()][['Hours']]

    # Expensive and cheap hours tomorrow list
    list_cheapest_hours_tomorrow = list(tomorrow_prices_df['Hour'][tomorrow_prices_df['Price Flag - Tier 1'] == 'Cheaper'])
    cheapest_hours_tomorrow = ','.join(list_cheapest_hours_tomorrow)

    list_expensive_hours_tomorrow = list(tomorrow_prices_df['Hour'][tomorrow_prices_df['Price Flag - Tier 1'] == 'Expensive'])
    expensive_hours_tomorrow = ','.join(list_expensive_hours_tomorrow)

    tomorrow = datetime.date.today() + datetime.timedelta(days=1)
    tomorrow_date = tomorrow.strftime('%d-%m-%Y')

    return f'Today is {today}','----------------------------------------------------------------------',f'Average price of today is: {average_today_price}€ per kwh',f'Lowest price of today is: {lowest_today_price}€ per kwh',f'Highest price of today is: {highest_today_price}€ per kwh','----------------------------------------------------------------------',f'Cheapest time of today was between {cheapest_today_time} hs',f'Expensive time of today was between {expensive_today_time} hs','----------------------------------------------------------------------',f'Tomorrow is {tomorrow_date}','----------------------------------------------------------------------',f'Average price for tomorrow will be: {average_price_tomorrow}€ per kwh',f'Lowest price for tomorrow will be: {lowest_price_tomorrow}€ per kwh',f'Highest price for tomorrow will be: {highest_price_tomorrow}€ per kwh','----------------------------------------------------------------------',f'The cheapest time for tomorrow will be {cheapest_hour_tomorrow} hs, while the cheapest hours for tomorrow will be: {cheapest_hours_tomorrow}',f'Most expensive time for tomorrow will be {expensive_hour_tomorrow} hs, while the most expensive hours for tomorrow will be: {expensive_hours_tomorrow}','----------------------------------------------------------------------',f'Today vs Tomorrow difference on the average price is {round(average_change_tomorrow_vs_today*100,2)}%',f'Today vs Tomorrow difference on the lowest price is {round(lowest_change_tomorrow_vs_today*100,2)}%',f'Today vs Tomorrow difference on the highest price is {round(highest_change_tomorrow_vs_today*100,2)}%'

In [9]:
today_tomorrow_comparison()

('Today is 04-05-2022',
 '----------------------------------------------------------------------',
 'Average price of today is: 0.2840€ per kwh',
 'Lowest price of today is: 0.25112€ per kwh',
 'Highest price of today is: 0.33504€ per kwh',
 '----------------------------------------------------------------------',
 'Cheapest time of today was between 01 - 02 hs',
 'Expensive time of today was between 21 - 22 hs',
 '----------------------------------------------------------------------',
 'Tomorrow is 05-05-2022',
 '----------------------------------------------------------------------',
 'Average price for tomorrow will be: 0.27194875€ per kwh',
 'Lowest price for tomorrow will be: 0.22875€ per kwh',
 'Highest price for tomorrow will be: 0.34489€ per kwh',
 '----------------------------------------------------------------------',
 'The cheapest time for tomorrow will be 14 hs, while the cheapest hours for tomorrow will be: 00,01,02,03,04,05,06,07,14,15,16,17',
 'Most expensive time for